## Installing libraries

In [10]:
# !pip install cohere
# !pip install langchain
# !pip install pypdf
# !pip install tiktoken
# !pip install chromadb
# !pip install bs4

## read url source file

In [11]:
def read_urls(file_path):
    '''Function to read URLs from a file'''
    with open(file_path, 'r') as file:
        urls = [url.strip() for url in file.readlines()]
    return urls

urls = read_urls("./source_url.txt")

## load the url source

In [12]:
from langchain.document_loaders import WebBaseLoader

# Specify the URL of the website you want to load
all_documents = []

# Load the website content for each URL and add it to the list
for url in urls:
    loader = WebBaseLoader(url)
    documents = loader.load()
    all_documents.extend(documents)

all_documents

[Document(page_content='\n405 Not Allowed\n\n405 Not Allowed\nAngie\n\n\n', metadata={'source': 'https://www.mql5.com/en/blogs/post/752096', 'title': '405 Not Allowed', 'language': 'No language found.'}),
 Document(page_content='\n405 Not Allowed\n\n405 Not Allowed\nAngie\n\n\n', metadata={'source': 'https://www.mql5.com/en/blogs/post/751762', 'title': '405 Not Allowed', 'language': 'No language found.'})]

## load document from local file

In [13]:
from langchain.document_loaders import TextLoader
loader = TextLoader('./source.txt')
all_documents = loader.load()

all_documents

[Document(page_content='Trades:\n744\nProfit Trades:\n490 (65.86%)\nLoss Trades:\n254 (34.14%)\nBest trade:\n57.45 USD\nWorst trade:\n-45.59 USD\nGross Profit:\n3 246.73 USD (344 065 pips)\nGross Loss:\n-1 705.90 USD (159 746 pips)\nMaximum consecutive wins:\n43 (101.14 USD)\nMaximal consecutive profit:\n124.46 USD (4)\nSharpe Ratio:\n0.19\nTrading activity:\n100.00%\nMax deposit load:\n8.24%\nLatest trade:\n3 days ago\nTrades per week:\n15\nAvg holding time:\n7 days \nRecovery Factor:\n12.02\nLong Trades:\n354 (47.58%)\nShort Trades:\n390 (52.42%)\nProfit Factor:\n1.90\nExpected Payoff:\n2.07 USD\nAverage Profit:\n6.63 USD\nAverage Loss:\n-6.72 USD\nMaximum consecutive losses:\n8 (-34.22 USD)\nMaximal consecutive loss:\n-75.82 USD (6)\nMonthly growth:\n5.89%\nAnnual Forecast:\n72.88%\nAlgo trading:\n73%\n\nGeneral Questions \nHow do I install expert advisors or indicators from the mql5.com market?\n\nKindly read my installation guide.\n\nHow do I update expert advisors or indicators f

## split the website content into smaller chunks

In [14]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
import os
import cohere

key = "urAN0hpVmnaPAW66RPqZQTKOSs46zd4EI93e1NNg"
co = cohere.Client(key)

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(all_documents)

os.environ["COHERE_API_KEY"] = key

## store vector

In [15]:
from langchain.embeddings import CohereEmbeddings
embeddings = CohereEmbeddings(cohere_api_key=key)
db = Chroma.from_documents(docs, embeddings)

## define prompt template

In [16]:
from langchain.prompts import PromptTemplate
prompt_template = """
you are a personal financial advisor here to help the client. 

Given the following financial context:

{context}

Question: {question}
As your financial advisor, I would recommend the following:
"""
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

## retrieve dara and output

In [17]:
from langchain.chains import RetrievalQA
from langchain.llms import Cohere

retriever = db.as_retriever()  # Assuming 'db' is your vector store

retrievalQA = RetrievalQA.from_llm(
    llm=Cohere(),
    retriever=retriever,
    prompt=prompt
)

## query 

In [20]:
query = "How do I update expert advisors or indicators from the mql5.com market to latest versions?"
retrievalQA.invoke(query)

{'query': 'How do I update expert advisors or indicators from the mql5.com market to latest versions?',
 'result': ' To update an expert advisor or indicator from the mql5.com market, follow the identical procedure for installing them, but instead of the "Install" button, click "Update" if the update is available. You don\'t have to pay for future updates as they are free. It\'s important to note that the update process may vary depending on the specific platform or software you are using. It\'s always best to consult the user manual or guidance provided by the platform developer for accurate and updated instructions. '}

In [21]:
query = "can you summarize my trading performance?"
retrievalQA.invoke(query)

{'query': 'can you summarize my trading performance?',
 'result': ' Given the provided trading information, here is a summary of your trading performance: \n\nYou have had 744 trades, and 65.86% of these trades have been profitable. This is an above-average ratio and is a positive sign. Maximal consecutive profit was $124.46, and maximal consecutive losses were -34.22 USD, meaning you have a consistent ability to recognize and ride profitable trends, and also minimize potential losses. \n\nThe gross profit of all trades is $3246.73 USD (344065 pips), and the gross loss is -1705.90 USD (159746 pips). This means that, on average, you allow yourself to make small consistent gains with strategic plays, and avoid overinvesting or placing too many risky bets. \n\nWith an average of 15 trades per week and an average holding time of 7 days, you trade with consistency and patience, two key aspects in investing. Based on this information, with the context of adequate trading activity and maximal

## invalid query

In [22]:
invalid_query = "where can i buy icecream?"
retrievalQA.invoke(invalid_query)

{'query': 'where can i buy icecream?',
 'result': " I appreciate your enthusiasm, but this ice cream question is beyond the scope of my expertise and abilities. I was designed solely to give financial advice and not recommendations on consumer products like ice cream. \n\nI'm sorry I can't be of more help with this. "}